In [1]:
import pandas as pd
import sqlite3
import json
import logging

### Reviews to DB

In [3]:
logging.basicConfig(filename='data_processing.log', level=logging.ERROR, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def filter_data(input_file, db_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS reviews (
        rating REAL,
        title TEXT,
        text TEXT,
        asin TEXT,
        parent_asin TEXT,
        user_id TEXT,
        timestamp INTEGER,
        helpful_vote INTEGER,
        verified_purchase BOOLEAN
    )
    ''')

    with open(input_file, 'r') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                data = json.loads(line)
                
                # Check if the timestamp is after 2018 (in milliseconds)
                if data['timestamp'] >= 1514764800000:  # 1514764800000 is Jan 1, 2018 in Unix ms
                    # Insert filtered data into the SQL table
                    cursor.execute('''
                    INSERT INTO reviews (rating, title, text, asin, parent_asin, user_id, timestamp, helpful_vote, verified_purchase)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        data['rating'], data['title'], data['text'],
                        data['asin'], data['parent_asin'], data['user_id'],
                        data['timestamp'], data['helpful_vote'],
                        data['verified_purchase']
                    ))
            except json.JSONDecodeError as e:
                logging.error(f"JSONDecodeError on line {line_number}: {str(e)}")
            except sqlite3.Error as e:
                logging.error(f"SQLite error on line {line_number}: {str(e)}")
            except Exception as e:
                logging.error(f"Unexpected error on line {line_number}: {str(e)}")

    conn.commit()
    conn.close()



In [ ]:
filter_data('Home_and_Kitchen.jsonl', 'Home_and_Kitchen_reviews.db')

### Metadata to DB

In [1]:
# Set up logging
logging.basicConfig(filename='data_processing.log', level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def save_matching_data_to_db(input_file, db_name, filtered_db):
    # Connect to the filtered database to get the list of parent_asin values
    filtered_conn = sqlite3.connect(filtered_db)
    filtered_cursor = filtered_conn.cursor()
    
    # Get distinct parent_asin from the filtered database
    filtered_cursor.execute("SELECT DISTINCT parent_asin FROM reviews")
    parent_asins = set(row[0] for row in filtered_cursor.fetchall())
    filtered_conn.close()

    # Connect to the target database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create a table for the product details
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        main_category TEXT,
        title TEXT,
        average_rating REAL,
        rating_number INTEGER,
        features TEXT,
        description TEXT,
        price REAL,
        store TEXT,
        categories TEXT,
        brand TEXT,
        material TEXT,
        color TEXT,
        capacity TEXT,
        style TEXT,
        pattern TEXT,
        care_instructions TEXT,
        unit_count TEXT,
        dimensions TEXT,
        num_items INTEGER,
        item_weight TEXT,
        best_sellers_rank TEXT,
        discontinued TEXT,
        date_first_available TEXT,
        parent_asin TEXT
    )
    ''')

    # Open the large dataset file
    with open(input_file, 'r') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                # Parse each line as JSON
                data = json.loads(line)

                # Check if the parent_asin matches the filtered set
                parent_asin = data.get('parent_asin', None)
                if parent_asin in parent_asins:
                    # Extract details from the JSON
                    main_category = data.get('main_category', None)
                    title = data.get('title', None)
                    average_rating = data.get('average_rating', None)
                    rating_number = data.get('rating_number', None)
                    features = json.dumps(data.get('features', []))
                    description = json.dumps(data.get('description', []))
                    price = data.get('price', None)
                    store = data.get('store', None)
                    categories = json.dumps(data.get('categories', []))
                    details = data.get('details', {})
                    brand = details.get('Brand', None)
                    material = details.get('Material', None)
                    color = details.get('Color', None)
                    capacity = details.get('Capacity', None)
                    style = details.get('Style', None)
                    pattern = details.get('Pattern', None)
                    care_instructions = details.get('Product Care Instructions', None)
                    unit_count = details.get('Unit Count', None)
                    dimensions = details.get('Product Dimensions', None)
                    num_items = details.get('Number of Items', None)
                    item_weight = details.get('Item Weight', None)
                    best_sellers_rank = json.dumps(details.get('Best Sellers Rank', {}))
                    discontinued = details.get('Is Discontinued By Manufacturer', None)
                    date_first_available = details.get('Date First Available', None)

                    # Insert the data into the SQL table
                    cursor.execute('''
                    INSERT INTO products (
                        main_category, title, average_rating, rating_number, features,
                        description, price, store, categories, brand, material, color,
                        capacity, style, pattern, care_instructions, unit_count, dimensions,
                        num_items, item_weight, best_sellers_rank, discontinued, date_first_available,
                        parent_asin
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        main_category, title, average_rating, rating_number, features,
                        description, price, store, categories, brand, material, color,
                        capacity, style, pattern, care_instructions, unit_count, dimensions,
                        num_items, item_weight, best_sellers_rank, discontinued, date_first_available,
                        parent_asin
                    ))
            except json.JSONDecodeError as e:
                logging.error(f"JSONDecodeError on line {line_number}: {str(e)}")
            except sqlite3.Error as e:
                logging.error(f"SQLite error on line {line_number}: {str(e)}")
            except Exception as e:
                logging.error(f"Unexpected error on line {line_number}: {str(e)}")

    # Commit changes and close the connection
    conn.commit()
    conn.close()


save_matching_data_to_db('meta_Home_and_Kitchen.jsonl', 'meta_Home_and_Kitchen.db', 'Home_and_Kitchen_reviews.db')


### Experiments

In [2]:
import sqlite3
import pandas as pd

def query_sample_rows(db_name, table_name, num_rows=5):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Query the specified number of rows from the table
    query = f"SELECT * FROM {table_name} LIMIT {num_rows}"
    cursor.execute(query)

    # Fetch the data and column names
    rows = cursor.fetchall()
    columns = [description[0] for description in cursor.description]

    # Convert to DataFrame for better readability
    df = pd.DataFrame(rows, columns=columns)

    # Close the connection
    conn.close()

    # Display the DataFrame
    # print(df)
    return df

# Usage example
meta_df = query_sample_rows('meta_Home_and_Kitchen.db', 'products', num_rows=50)


In [ ]:
meta_df#.columns#['categories']

In [2]:
def query_db(query,db_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Query the specified number of rows from the table
    # query = f"SELECT * FROM {table_name} LIMIT {num_rows}"
    cursor.execute(query)

    # Fetch the data and column names
    rows = cursor.fetchall()
    columns = [description[0] for description in cursor.description]

    # Convert to DataFrame for better readability
    df = pd.DataFrame(rows, columns=columns)

    # Close the connection
    conn.close()

    # Display the DataFrame
    # print(df)
    return df



In [5]:
query = """SELECT 
            *
            FROM products
            LIMIT 5
        """
query_db(query,'meta_Home_and_Kitchen.db').to_csv('meta_Home_and_Kitchen_sample.csv',index=False)

In [6]:
query = """SELECT 
            *
            FROM reviews
            LIMIT 5
        """
query_db(query,'Home_and_Kitchen_reviews.db').to_csv('reviews_Home_and_Kitchen_sample.csv',index=False)

In [5]:
# Usage example
query = """SELECT 
            DISTINCT main_category, 
            COUNT(parent_asin) AS product_count
            FROM products
            GROUP BY(main_category)


        """
meta_df = query_db(query,'meta_Home_and_Kitchen.db')


In [ ]:
meta_df

In [ ]:
query = """SELECT 
            *
            FROM products
            WHERE main_category="Health & Personal Care"
            -- GROUP BY(main_category)
        """
temp_df = query_db(query,'meta_Home_and_Kitchen.db')
temp_df

In [ ]:

print(len(temp_df["store"][:].unique()))

In [ ]:
query = """SELECT 
            DISTINCT store,
            COUNT(parent_asin) AS product_count
            FROM products
            WHERE main_category="Amazon Home"
            GROUP BY(store)
            ORDER BY product_count DESC
            LIMIT 100
        """
query_db(query,'meta_Home_and_Kitchen.db')
# temp_df.to_csv("store_product_count_from_metadata.csv")

In [20]:
query = """SELECT *
            FROM products
            WHERE title LIKE "%office desk,%" 
        """
temp_df = query_db(query,'meta_Home_and_Kitchen.db')
# temp_df.to_csv("store_product_count_from_metadata.csv")

In [ ]:
temp_df['features'].loc[0]#0['style']

In [ ]:
query = """
    SELECT *
            FROM reviews
    WHERE parent_asin="B07TP32NF7"
    """
query_db(query,'Home_and_Kitchen_reviews.db')



In [ ]:
import sqlite3
from datetime import datetime

def query_db(db_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    cursor.execute('SELECT MIN(timestamp), MAX(timestamp) FROM reviews')
    min_max_result = cursor.fetchone()
    
    if min_max_result[0] is not None and min_max_result[1] is not None:
        min_date = datetime.fromtimestamp(min_max_result[0] / 1000)
        max_date = datetime.fromtimestamp(min_max_result[1] / 1000)
        print(f"Minimum Timestamp: {min_max_result[0]} ({min_date})")
        print(f"Maximum Timestamp: {min_max_result[1]} ({max_date})")
    else:
        print("No data found in the database.")

    cursor.execute('SELECT COUNT(*) FROM reviews')
    total_rows = cursor.fetchone()[0]
    print(f"Total number of rows: {total_rows}")

    conn.close()

query_db('Home_and_Kitchen_reviews.db')

In [ ]:
import sqlite3
import pandas as pd

def query_aggregated_data(db_name):
    conn = sqlite3.connect(db_name)
    
    query = """
    WITH Aggregated AS (
        SELECT
            parent_asin,
            MIN(timestamp) AS review_start,
            MAX(timestamp) AS review_end,
            COUNT(timestamp) AS review_count
        FROM reviews
        GROUP BY parent_asin
    ),
    DateComponents AS (
        SELECT
            parent_asin,
            review_start,
            review_end,
            review_count,
            strftime('%Y', datetime(review_start / 1000, 'unixepoch')) AS review_start_year,
            strftime('%Y', datetime(review_end / 1000, 'unixepoch')) AS review_end_year,
            strftime('%m', datetime(review_start / 1000, 'unixepoch')) AS review_start_month,
            strftime('%m', datetime(review_end / 1000, 'unixepoch')) AS review_end_month,
            date(datetime(review_start / 1000, 'unixepoch')) AS review_start_date,
            date(datetime(review_end / 1000, 'unixepoch')) AS review_end_date
        FROM Aggregated
    )
    SELECT *
    FROM DateComponents
    WHERE 
        review_start_year >= '2018' AND
        review_start_year <= '2020' AND
        review_end_year > '2020' AND
        review_count > 100
    ORDER BY review_end_date DESC
    """
    
    aggregated_data = pd.read_sql_query(query, conn)
    
    conn.close()

    print("Total reviews:", aggregated_data['review_count'].sum())
    return aggregated_data

aggregated_data_sql_df = query_aggregated_data('Home_and_Kitchen_reviews.db')


In [ ]:
aggregated_data_sql_df

In [ ]:
import sqlite3
import pandas as pd

def query_aggregated_data(db_name):
    conn = sqlite3.connect(db_name)
    
    query = """
WITH Aggregated AS (
    SELECT
        parent_asin,
        MIN(timestamp) AS review_start,
        MAX(timestamp) AS review_end,
        COUNT(timestamp) AS review_count
    FROM reviews
    GROUP BY parent_asin
),
DateComponents AS (
    SELECT
        parent_asin,
        review_start,
        review_end,
        review_count,
        strftime('%Y', datetime(review_start / 1000, 'unixepoch')) AS review_start_year,
        strftime('%Y', datetime(review_end / 1000, 'unixepoch')) AS review_end_year,
        strftime('%m', datetime(review_start / 1000, 'unixepoch')) AS review_start_month,
        strftime('%m', datetime(review_end / 1000, 'unixepoch')) AS review_end_month,
        date(datetime(review_start / 1000, 'unixepoch')) AS review_start_date,
        date(datetime(review_end / 1000, 'unixepoch')) AS review_end_date
    FROM Aggregated
    WHERE 
        review_start_year >= '2018' AND
        review_start_year <= '2020' AND
        review_end_year > '2020' AND
        review_count > 100
),
RatingCounts AS (
    SELECT 
        asin,
        parent_asin,
        COUNT(CASE WHEN rating = 1 THEN 1 END) AS rating_1_count,
        COUNT(CASE WHEN rating = 2 THEN 1 END) AS rating_2_count,
        COUNT(CASE WHEN rating = 3 THEN 1 END) AS rating_3_count,
        COUNT(CASE WHEN rating = 4 THEN 1 END) AS rating_4_count,
        COUNT(CASE WHEN rating = 5 THEN 1 END) AS rating_5_count,
        COUNT(asin) AS total_count
    FROM reviews
    GROUP BY asin, parent_asin
)
SELECT 
    d.parent_asin,
    r.asin,
    d.review_start,
    d.review_end,
    d.review_start_date,
    d.review_end_date,
    d.review_count,
    r.rating_1_count,
    r.rating_2_count,
    r.rating_3_count,
    r.rating_4_count,
    r.rating_5_count,
    r.total_count,
    ROUND(CAST(r.rating_1_count AS FLOAT) / r.total_count * 100, 2) AS neg_review_percentage,
    ROUND(CAST(r.rating_5_count AS FLOAT) / r.total_count * 100, 2) AS pos_review_percentage
FROM 
    DateComponents d
JOIN 
    RatingCounts r
ON 
    d.parent_asin = r.parent_asin
ORDER BY 
    d.review_end_date DESC;
    """
    
    aggregated_data = pd.read_sql_query(query, conn)
    
    conn.close()

    print("Total reviews:", aggregated_data['review_count'].sum())
    return aggregated_data

aggregated_data_sql_df = query_aggregated_data('Home_and_Kitchen_reviews.db')


In [9]:
aggregated_data_sql_df.to_csv("Home_and_Kitchen_rating_distribution.csv")

In [9]:
def get_rating_distribution(db_name):
    # Connect to SQLite database
    conn = sqlite3.connect(db_name)
    
    # SQL query to get rating distribution grouped by asin and rating
    query = """
SELECT 
    parent_asin,asin,
    COUNT(CASE WHEN rating = 1 THEN 1 END) AS rating_1_count,
    COUNT(CASE WHEN rating = 2 THEN 1 END) AS rating_2_count,
    COUNT(CASE WHEN rating = 3 THEN 1 END) AS rating_3_count,
    COUNT(CASE WHEN rating = 4 THEN 1 END) AS rating_4_count,
    COUNT(CASE WHEN rating = 5 THEN 1 END) AS rating_5_count,
    COUNT(asin) AS total_count
FROM 
    reviews
GROUP BY 
    parent_asin,asin
ORDER BY 
    asin;
    """
    
    # Execute query and load results into a DataFrame
    rating_distribution = pd.read_sql_query(query, conn)
    
    # Close connection
    conn.close()

    # Display the DataFrame
    # print(rating_distribution)
    return rating_distribution

rating_distribution = get_rating_distribution('Home_and_Kitchen_reviews.db')


In [ ]:
rating_distribution

In [ ]:
import sqlite3
import pandas as pd
query = """SELECT COUNT(*)
            FROM reviews
            -- LIMIT 20
        """
query_db(query,'Home_and_Kitchen_reviews.db')


In [4]:
# 1514764800000
h_k_5_core_df = pd.read_csv('Home_and_Kitchen_5_core.csv') 

In [ ]:
h_k_5_core_df.shape

In [6]:
h_k_5_core_df = h_k_5_core_df[h_k_5_core_df['timestamp']>=1514764800000]

In [ ]:
h_k_5_core_df.shape

In [ ]:
len(set(h_k_5_core_df['user_id']))

In [27]:
def query_db_with_user_ids(user_ids, db_name):
    # Convert the list of user_ids to a format suitable for SQL IN clause
    user_ids_str = ','.join([f"'{user_id}'" for user_id in user_ids])
    
    # SQL query to select records where user_id is in the provided list
    query = f"""
            SELECT *
            FROM reviews
            WHERE user_id IN ({user_ids_str})
            """
    # print(query)
    # Call the existing query_db function to execute the query
    return query_db(query, db_name)

# Assuming you already have your DataFrame
# user_ids = set(h_k_5_core_df['user_id'])#.tolist()

# Query the database
# result_df = query_db_with_user_ids(user_ids, 'Home_and_Kitchen_reviews.db')

# Display the resulting DataFrame
# print(result_df)


In [28]:
result_df = query_db_with_user_ids(user_ids, 'Home_and_Kitchen_reviews.db')


In [ ]:
user_ids = h_k_5_core_df['user_id'].unique()#.tolist()

temp = user_ids[:10]
query = f"""
        SELECT *
        FROM reviews
        WHERE user_id IN ({temp})
        """
print(query)


In [15]:
user_ids_str = ','.join([f"'{user_id}'" for user_id in temp])


In [ ]:
result_df

In [2]:
result_df = pd.read_csv('reviews_filtered_5core.csv')

In [ ]:
result_df.shape

In [ ]:
result_df.groupby('parent_asin').first()

In [ ]:
result_df[:1]

In [20]:
def calculate_review_distribution(df, pos_range=(5, 5), neg_range=(1, 1)):
    # Group by 'parent_asin' and 'asin', then calculate rating counts (1.0 to 5.0)
    asin_parent_group = df.groupby(['parent_asin', 'asin'])['rating'].value_counts().unstack(fill_value=0)

    # Rename columns for clarity (rating_1, rating_2, ..., rating_5)
    asin_parent_group.columns = [f'rating_{int(rating)}' for rating in asin_parent_group.columns]

    # Calculate total ratings for each asin (sum of all rating columns)
    asin_parent_group['total_ratings_asin'] = asin_parent_group.sum(axis=1)

    # Calculate total positive and negative ratings based on the provided ranges
    asin_parent_group['total_pos_asin'] = asin_parent_group[[f'rating_{i}' for i in range(pos_range[0], pos_range[1] + 1)]].sum(axis=1)
    asin_parent_group['total_neg_asin'] = asin_parent_group[[f'rating_{i}' for i in range(neg_range[0], neg_range[1] + 1)]].sum(axis=1)

    # Calculate positive and negative percentages for each asin
    asin_parent_group['pos_per_asin'] = asin_parent_group['total_pos_asin'] / asin_parent_group['total_ratings_asin'] * 100
    asin_parent_group['neg_per_asin'] = asin_parent_group['total_neg_asin'] / asin_parent_group['total_ratings_asin'] * 100

    # Group by 'parent_asin' to calculate totals, including positive and negative ratings for each parent_asin
    parent_group = asin_parent_group.groupby('parent_asin').sum()

    # Calculate total ratings for the entire parent_asin (sum of all ratings for all asins under that parent_asin)
    parent_group['total_ratings_parent_asin'] = parent_group[['rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5']].sum(axis=1)

    # Calculate total positive and negative ratings for each parent_asin
    parent_group['total_pos_parent_asin'] = parent_group[[f'rating_{i}' for i in range(pos_range[0], pos_range[1] + 1)]].sum(axis=1)
    parent_group['total_neg_parent_asin'] = parent_group[[f'rating_{i}' for i in range(neg_range[0], neg_range[1] + 1)]].sum(axis=1)

    # Calculate positive and negative percentages for each parent_asin
    parent_group['pos_per_parent_asin'] = parent_group['total_pos_parent_asin'] / parent_group['total_ratings_parent_asin'] * 100
    parent_group['neg_per_parent_asin'] = parent_group['total_neg_parent_asin'] / parent_group['total_ratings_parent_asin'] * 100

    # Merge asin-level and parent-level results for final display, including total ratings and totals for pos/neg
    result = asin_parent_group.merge(parent_group[['total_ratings_parent_asin', 'pos_per_parent_asin', 'neg_per_parent_asin', 'total_pos_parent_asin', 'total_neg_parent_asin']],
                                     left_on='parent_asin', right_index=True, how='left')

    return result

review_distribution = calculate_review_distribution(result_df)


In [ ]:
sorted_distribution = review_distribution.sort_values(by='total_ratings_parent_asin',ascending=False)#.get('parent_asin')
sorted_distribution

In [ ]:
sorted_distribution['total_neg_asin'].sum(),sorted_distribution['total_pos_asin'].sum()

In [30]:
sorted_parent_asins = sorted_distribution.index.get_level_values('parent_asin').unique()#.tolist()

In [ ]:
sorted_parent_asins

In [ ]:
# review_distribution

review_distribution.loc[review_distribution.index.get_level_values('parent_asin')  == 'B08XYD548C']

In [ ]:
result_df[result_df['parent_asin']=='B00U8QEXBS']#['verified_purchase'].value_counts()